<a href="https://colab.research.google.com/github/Rishikesh-Guptha/amazonwebscrapping/blob/main/AmazonWebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install bs4

In [3]:
pip install pandas

In [4]:
pip install requests

In [51]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [19]:
url="https://www.amazon.in/s?k=phones"
HEADER={
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}                                                 #using headers to make the request seem like legit from the browser
response=requests.get(url,headers=HEADER)         #sending request to that url

In [20]:
type(response.content)

bytes

In [22]:
page=BeautifulSoup(response.content,"html.parser")      # This is done to convert the content which is in bytes to html format

In [28]:
main_link=page.find("span",attrs={"class":"rush-component s-latency-cf-section"})
# print(main_links)
links=main_link.find_all("a",attrs={"class":"a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"})
print(len(links))

16


In [29]:
link_list=[]
for link in links:
    link_list.append("https://amazon.in"+link.get("href"))
print(link_list[0])

https://amazon.in/Samsung-Storage-Display-Charging-Security/dp/B0DFY3XCB6/ref=sr_1_1?dib=eyJ2IjoiMSJ9.YFHwfIh7GPJJ09bm1bBIDyerTatsbYLfbLTzZOtIOI_PhS-3_IY2Fs73Ku6LbRlMMUIRqYtt8ih0RwOSY5AjE7djkg1a8IEVT1GLkiLsRtarn7IhhUQFw0LXgEHRSW_jqp9P0sCf4-dGiNL_Smsx5mJrsS3trDKgOgnavvJT3MZc85I8DY4CHSHRtWU3Pu76mUrv-7qlXFOHyK-0N0x2z2NUkz0e-fOUxtCuMey01Ts.8rkFCHy0tubhJvmvDqhKGKycmpGbRjRS3HDBNZnsCSY&dib_tag=se&keywords=phones&qid=1733588145&sr=8-1


In [71]:
def get_title(soup):
    try:
        title=soup.find("span",attrs={"id":"productTitle"}).text.strip()
        brand=''
        model=''
        match = re.match(r"(\w+)\s+([a-zA-Z]+\s*\d+)", title)
        if match:
            brand = match.group(1).strip()  # Extracts ex: "Apple"
            model = match.group(2).strip()  # Extracts ex: "iPhone 13"

    except AttributeError:
        title=""
    return [brand,model]

In [31]:
def get_price(soup):
    try:
        price=soup.find("span",attrs={"class":"a-price-whole"}).text.strip()
    except AttributeError:
        price=""
    return price

In [64]:
def get_rating(soup):
    try:
        rating=soup.find("span",attrs={"id":"acrPopover"}).find("span",attrs={"class":"a-size-base a-color-base"}).text.strip()
    except AttributeError:
        rating=""
    return rating

In [33]:
def get_memory(soup):
    try:
        memory=soup.find("span",attrs={"class":"selection"}).text.strip()
    except AttributeError:
        memory=""
    return memory

In [34]:
def availability(soup):
  try:
    available=soup.find("div",attrs={"id":"a-size-medium a-color-success"}).find("span").text.strip()
  except AttributeError:
    available=""
  return available

In [78]:
phone_specs={"Name":[],"Brand":[],"Price":[],"Rating":[],"Memory":[],"Availability":[]}
for link in link_list:
    product_response=requests.get(link,headers=HEADER)
    product=BeautifulSoup(product_response.content,"html.parser")
    brand_model=get_title(product)
    if brand_model and len(brand_model[0])>1 and len(brand_model[1])>1:
      phone_specs["Name"].append(brand_model[0])
      phone_specs["Brand"].append(brand_model[1])
      phone_specs["Price"].append(get_price(product))
      phone_specs["Rating"].append(get_rating(product))
      phone_specs["Memory"].append(get_memory(product))
      phone_specs["Availability"].append(availability(product))
    else:
      continue




Redmi A4
Redmi A4
POCO M6
Redmi A4
Lava Yuva 3
POCO C61
Redmi A4


In [79]:
print(len(phone_specs["Name"]))
print(len(phone_specs["Brand"]))

7
7


In [81]:
amazon_phone_df=pd.DataFrame.from_dict(phone_specs)
amazon_phone_df=amazon_phone_df.dropna(subset=['Name'])
amazon_phone_df.to_csv("amazon_phones.csv",header=True,index=False)